<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/2_TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트 데이터 수집 및 TF/IDF 분석
 - 무료로 수집가능한 Corpus : http://www.nltk.org/howto/corpus.html

* 말뭉치(Corpus)의 사용사례
 - NLTK(Natural Language Toolkit)패키지는 실무 및 연구용을 활용이 가능한 자연어 처리 패키지
 - 말뭉치, 토큰 생성, 형태소 분석, 품사 태깅에 사용할 수 있음

* NLTK 말뭉치 유형 예시
 - brown : 약 100만 단어의 묶음 샘플로 브라운대학교에서 표준적 영어문장을 정리함
 - gutenberg : 저작권이 말소된 문학작품의 샘플
 - names : 8000개의 남성과 여성의 이름 리스트
 - words : 가장 빈번하게 사용하는 영어 단어 23만 5000개
 - stopwords : 14개 유형의 언어가 지원되며, 가장 일반적인 불용어(stop word) 리스트

# NLTK에서 제공되는 말뭉치(Corpus)를 조회하고, "영화리뷰"와 관련된 말뭉치를 다운

In [27]:
import nltk.corpus

print(dir(nltk.corpus))
nltk.download("movie_reviews", quiet=True)

['AlignedCorpusReader', 'AlpinoCorpusReader', 'BCP47CorpusReader', 'BNCCorpusReader', 'BracketParseCorpusReader', 'CHILDESCorpusReader', 'CMUDictCorpusReader', 'CategorizedBracketParseCorpusReader', 'CategorizedCorpusReader', 'CategorizedPlaintextCorpusReader', 'CategorizedSentencesCorpusReader', 'CategorizedTaggedCorpusReader', 'ChasenCorpusReader', 'ChunkedCorpusReader', 'ComparativeSentencesCorpusReader', 'ConllChunkCorpusReader', 'ConllCorpusReader', 'CorpusReader', 'CrubadanCorpusReader', 'DependencyCorpusReader', 'EuroparlCorpusReader', 'FramenetCorpusReader', 'IEERCorpusReader', 'IPIPANCorpusReader', 'IndianCorpusReader', 'KNBCorpusReader', 'LazyCorpusLoader', 'LinThesaurusCorpusReader', 'MTECorpusReader', 'MWAPPDBCorpusReader', 'MacMorphoCorpusReader', 'NKJPCorpusReader', 'NPSChatCorpusReader', 'NombankCorpusReader', 'NonbreakingPrefixesCorpusReader', 'OpinionLexiconCorpusReader', 'PPAttachmentCorpusReader', 'PanLexLiteCorpusReader', 'PanlexSwadeshCorpusReader', 'Pl196xCorpusRe

True

# 영화리뷰 코퍼스의 파일 리스트 확인

In [28]:
fileList = nltk.corpus.movie_reviews.fileids()
len(fileList)
fileList[:5]

['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt']

* 전체 영화리뷰 목록 중 부정적 리뷰의견 조회

In [29]:
review_txt1 = nltk.corpus.movie_reviews.raw("neg/cv000_29416.txt")
print(review_txt1[:4043])

plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
what's the deal ? 
watch the movie and " sorta " find out . . . 
critique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . 
which is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn't snag this one correctly . 
they seem to have taken this pretty neat concept , but executed it terribly . 
so what are the problems with the movie ? 
well , its main problem is that it's simply too jumbled . 
it starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience member , have no id

# 모델기반의 클러스터링을 위해 TF-IDF를 사용하는 예제
 - Python을 활용하여 TF 계산

In [30]:
# 말뭉치(Corpus)를 통합하여 단어집합 구성
import pandas as pd
import math

# 분석을 위해 수집된 말뭉치(Corpus)를 입력
text_corpus = ['여기는 나의 작업 공간이며 나는 작업 중이다', '나는 그 작업 공간에 있다', '내일도 퇴사준비 작업 예정이다' ]

# 각각의 문서에서 단어를 추출
text1_list = text_corpus[0].split(" ")
text2_list = text_corpus[1].split(" ")
text3_list = text_corpus[2].split(" ")

# 공통적으로 중복된 단어를 제거, 제거전 15개의 단어
doc_set= set(text1_list).union(set(text2_list)).union(set(text3_list))

# 4개의 중복 단어를 제거하고 14개의 단어집합(Set) 구성
print(doc_set)

{'나는', '예정이다', '나의', '있다', '공간이며', '작업', '공간에', '그', '내일도', '중이다', '퇴사준비', '여기는'}


In [31]:
print(text_corpus[0])

여기는 나의 작업 공간이며 나는 작업 중이다


 - doc_set의 전체 단어집합을 딕셔너리를 생성하며 값은 모두 0으로 저장하고, 빈도 추가

In [32]:
# dict.fromkeys는 키가 들어있는 리스트를 넣으면 딕셔너리를 생성
text1_dict = dict.fromkeys(doc_set, 0)
text2_dict = dict.fromkeys(doc_set, 0)
text3_dict = dict.fromkeys(doc_set, 0)

# 딕셔너리는 키-값을 쌍으로 관리 용이
print(text1_dict)
print(text1_dict['공간이며'])

{'나는': 0, '예정이다': 0, '나의': 0, '있다': 0, '공간이며': 0, '작업': 0, '공간에': 0, '그': 0, '내일도': 0, '중이다': 0, '퇴사준비': 0, '여기는': 0}
0


In [33]:
# 문서별로 해당 단어가 확인되면 리스트의 값을 1개씩 증가
for text_word in text1_list:
    text1_dict[text_word]+=1

for text_word in text2_list:
    text2_dict[text_word]+=1

for text_word in text3_list:
    text3_dict[text_word]+=1

print(text1_dict)

{'나는': 1, '예정이다': 0, '나의': 1, '있다': 0, '공간이며': 1, '작업': 2, '공간에': 0, '그': 0, '내일도': 0, '중이다': 1, '퇴사준비': 0, '여기는': 1}


In [34]:
# 문서별 단어빈도가 확인 리스트를 하나의 데이터프레임으로 합산
pd.DataFrame([text1_dict, text2_dict, text3_dict])

,나는,예정이다,나의,있다,공간이며,작업,공간에,그,내일도,중이다,퇴사준비,여기는
0,1,0,1,0,1,2,0,0,0,1,0,1
1,1,0,0,1,0,1,1,1,0,0,0,0
2,0,1,0,0,0,1,0,0,1,0,1,0


* 문서 중에서의 출현 빈도를 출현비율로 변경
  - TF는 각 문서에서의 해당 단어의 빈도로 계산
  - 해당 단어 개수 / 해당 문서 전체 단어 개수

In [35]:
# TF를 구하기 위한 함수 정의
def getTF(temp_dic, temp_list):
    tfDict = {}
    wordCnt = len(temp_list)
    for word_str, count in temp_dic.items():
        tfDict[word_str] = count/float(wordCnt)
        #print(word_str)
        #print(count)
    return tfDict

# TF함수를 사용하여 각 문서 단어별 TF를 계산
text1_tfDict = getTF(text1_dict, text1_list)
text2_tfDict = getTF(text2_dict, text2_list)
text3_tfDict = getTF(text3_dict, text3_list)

print(text1_dict)
print(text1_list)

# 전체 문서를 표형태의 데이터프레임으로 표현
tf_df= pd.DataFrame([text1_tfDict, text2_tfDict, text3_tfDict])
tf_df

{'나는': 1, '예정이다': 0, '나의': 1, '있다': 0, '공간이며': 1, '작업': 2, '공간에': 0, '그': 0, '내일도': 0, '중이다': 1, '퇴사준비': 0, '여기는': 1}
['여기는', '나의', '작업', '공간이며', '나는', '작업', '중이다']


,나는,예정이다,나의,있다,공간이며,작업,공간에,그,내일도,중이다,퇴사준비,여기는
0,0.142857,0.00,0.142857,0.0,0.142857,0.285714,0.0,0.0,0.00,0.142857,0.00,0.142857
1,0.200000,0.00,0.000000,0.2,0.000000,0.200000,0.2,0.2,0.00,0.000000,0.00,0.000000
2,0.000000,0.25,0.000000,0.0,0.000000,0.250000,0.0,0.0,0.25,0.000000,0.25,0.000000


* Python의 활용하여 IDF 계산
  - IDF는 전체 문서 중 특정 문서에서만 출현하는 단어를 추출
  - log(전체 문서 수 / 해당 단어가 포함된 문서 수)

In [36]:
# IDF를 구하기 위한 함수 정의
def getIDF(temp_dict):
    tmp = {}
    totalCnt = len(temp_dict)

    tmp = dict.fromkeys(temp_dict[0].keys(), 0)
    for doc in temp_dict:
        for strword, cntword in doc.items():
            if cntword > 0:
                tmp[strword] += 1

    for sWord, intCnt in tmp.items():
        tmp[sWord] = math.log10(totalCnt / float(intCnt)) + 1

    return tmp

# 전체문서를 입력 값으로 getIDF함수를 통해서 계산
idf_dict = getIDF([text1_dict, text2_dict, text3_dict])
idf_dict


{'나는': 1.1760912590556813,
 '예정이다': 1.4771212547196624,
 '나의': 1.4771212547196624,
 '있다': 1.4771212547196624,
 '공간이며': 1.4771212547196624,
 '작업': 1.0,
 '공간에': 1.4771212547196624,
 '그': 1.4771212547196624,
 '내일도': 1.4771212547196624,
 '중이다': 1.4771212547196624,
 '퇴사준비': 1.4771212547196624,
 '여기는': 1.4771212547196624}

* Python의 활용하여 TF-IDF 계산
  - TF-IDF는 TF와 IDF를 합산하여 계산  

In [37]:
# TF-IDF를 구하기 위한 함수 정의
def getTFIDF(temp_dict, temp_idf):
    tmp = {}
    for strword, cntword  in temp_dict.items():
        tmp[strword] = cntword*temp_idf[strword]
    return tmp

# 해당 문서의 TF단어와 IDF로 계산된 전체 단어와 합산
text1_idfdict = getTFIDF(text1_tfDict, idf_dict)
text2_idfdict = getTFIDF(text2_tfDict, idf_dict)
text3_idfdict = getTFIDF(text3_tfDict, idf_dict)

# 데이터 프레임으로 전환
tfidf_df = pd.DataFrame([text1_idfdict, text2_idfdict,text3_idfdict ])
tfidf_df

,나는,예정이다,나의,있다,공간이며,작업,공간에,그,내일도,중이다,퇴사준비,여기는
0,0.168013,0.00000,0.211017,0.000000,0.211017,0.285714,0.000000,0.000000,0.00000,0.211017,0.00000,0.211017
1,0.235218,0.00000,0.000000,0.295424,0.000000,0.200000,0.295424,0.295424,0.00000,0.000000,0.00000,0.000000
2,0.000000,0.36928,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.36928,0.000000,0.36928,0.000000


# sklearn을 활용하여 TF, DF, TF-IDF 계산
  - sklearn의 TF-IDF는 한글자는 자동으로 제외됨
  - IDF의 '0'반환을 최소화하기 위해 가중치 +1이 적용
  - 불용어처리, 최소/최대빈도, 분석기준(단어, 글자) 정의 및 다양한 파라미터 제공

In [38]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 분석을 위해 수집된 말뭉치(Corpus)를 입력
text_corpus = ['여기는 나의 작업 공간이며 나는 작업 중이다', '나는 그 작업 공간에 있다', '내일도 작업 예정이다' ]

# 입력된 문서와 분석결과를 직관적으로 분석하기 위해 전환처리
df_text_corpus  = pd.DataFrame([text_corpus]).T
# text_corpus을 컬럼명으로 지정
df_text_corpus.columns=['text_corpus']

# 사이키런의 TfidfVectorizer를 생성하고 코퍼스를 전환처리
tfidf_o = TfidfVectorizer()
tfidf_dense = tfidf_o.fit_transform(df_text_corpus['text_corpus']).todense()
words_list = tfidf_o.get_feature_names_out()

# 데이터 프레임으로 전환
df_text_corpus = df_text_corpus.join(pd.DataFrame(tfidf_dense, columns=words_list))
df_text_corpus

,text_corpus,공간에,공간이며,나는,나의,내일도,여기는,예정이다,있다,작업,중이다
0,여기는 나의 작업 공간이며 나는 작업 중이다,0.000000,0.409146,0.311166,0.409146,0.000000,0.409146,0.000000,0.000000,0.483296,0.409146
1,나는 그 작업 공간에 있다,0.584483,0.000000,0.444514,0.000000,0.000000,0.000000,0.000000,0.584483,0.345205,0.000000
2,내일도 작업 예정이다,0.000000,0.000000,0.000000,0.000000,0.652491,0.000000,0.652491,0.000000,0.385372,0.000000


# 모델기반의 클러스터링을 위한 k-Means 적용예제(1)
* K-Means
 - 주어진 데이터를 K개의 클러스터로 묶는 알고리즘
 - 많은 양의 텍스트 데이터를 분류하여 적정 코퍼스구성을 위해 사용
* K-Means 적용시 고려사항
 - 중심점에 따라 클러스터가 민감하게 변함
 - k의 개수를 선정하는 문제 존재
 - 적정 k(클러스터링 개수)의 산정은 Elbow, Silhouette Method 등 활용

 - 경제와 영화 관련 뉴스 제목을 군집의 개수 2개로 구분

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# 분석에 필요한 말뭉치(corpus) 정의
documents = ["한은 저유가 당분간 계속 주요국 물가 하방 압력",
             "경총 국민 절반 이상이 동결 요구",
             "경제 바닥 통과 기대 강화",
             "도시 가스 물가 향상 기여",
             "예측 불가능한 하늘에 도전했던 사람들 영화",
             "전주 국제 영화 상영작",
             "백상 예술 대상 영화 감독 봉준호",
             "한번도 본적 없는 새로운 액션 영화"]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents)

# k-means++ 알고리즘을 적용, k개는 2개로 선정
# init은 초기화 방법으로 random과 k-means++ 선택 가능
# max_iter는 최대 반혹 횟수
# n_init는 초기 중심 위치 시도 횟수, 기본은 10으로 사용
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

KMeans(max_iter=100, n_clusters=2, n_init=10)

 - 2개의 그룹의 주요 키워드와 새로운 뉴스제목의 그룹을 예측분류

In [40]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["서울 환경 영화 공동 주최"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["세계 경제 물가 침체 예상"])
prediction = model.predict(Y)
print(prediction)

Top terms per cluster:
Cluster 0:
 영화
 상영작
 전주
 국제
 도전했던
 대상
 백상
 본적
 봉준호
 사람들
Cluster 1:
 물가
 향상
 기여
 도시
 가스
 통과
 강화
 바닥
 기대
 경제


Prediction
[0]
[1]
